In [14]:
import pandas as pd
import numpy as np
import os


def process(text: str):
    from parse import compile
    from string import punctuation

    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans("", "", punctuation)).lower())
    return " ".join(out_list)

df = pd.read_json("ParlaSpeech-HR.v1.0.jsonl", orient="records", lines=True)
df = df.rename(columns={"path":"hashname"})
df = df.loc[df.split.isin("test,dev".split(",")), :]

df["sentence"] = df.words.apply(" ".join).apply(process)
df["path"] = df.hashname.apply(lambda s: os.path.join(transferdir, s))
df.head(2)

,hashname,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times,utterance_id_start,utterance_id_end,speaker_info,split,norm_words_edited,sentence,path
1,Ki_SnDM_EkQ_2917.58-2937.5.wav,"18 10 2018 - 9. sjednica, 9. saziv [Ki_SnDM_Ek...",2917.58,2937.50,"[izraelska, tvrtka, prodaje, avione, Hrvatskoj...","[0, 0.65, 1.04, 1.54, 1.92, 2.51, 3.11, 3.5700...","[izraelska, tvrtka, prodaje, avione, hrvatskoj...","[0, 0.65, 1.04, 1.54, 1.92, 2.51, 3.11, 3.5700...",ParlaMint-HR_S09.u4267,ParlaMint-HR_S09.u4267,"{'Speaker_role': 'Regular', 'Speaker_type': 'M...",dev,NaN,izraelska tvrtka prodaje avione hrvatskoj dire...,transfer/Ki_SnDM_EkQ_2917.58-2937.5.wav
2,XguZsDKdRh4_13797.59-13811.74.wav,"20 1 2017 - 3. sjednica, 9. saziv [XguZsDKdRh4...",13797.59,13811.74,"[jučer, trećem,, sutra, ili, prekosutra, četvr...","[0, 0.43, 1.0, 1.4, 1.52, 2.01, 2.55, 3.12, 3....","[jučer, trećem, sutra, ili, prekosutra, četvrt...","[0, 0.43, 1.0, 1.4, 1.52, 2.01, 2.55, 3.12, 3....",ParlaMint-HR_S03.u9702,ParlaMint-HR_S03.u9702,"{'Speaker_role': 'Regular', 'Speaker_type': 'M...",dev,NaN,jučer trećem sutra ili prekosutra četvrtoj oso...,transfer/XguZsDKdRh4_13797.59-13811.74.wav


In [15]:
import os
from tqdm.auto import tqdm
transferdir = "transfer/"

if all([os.path.exists(
                        os.path.join(
                            transferdir,
                            hashname
                        )
                    )  for hashname in tqdm(df.hashname) ]):
    print("All files exist!")
else:
    for hashname in tqdm(df.hashname):
        path = os.path.join(transferdir, hashname)
        if not os.path.exists(path):
            print(f"File {hashname} doesn't exist!")

  0%|          | 0/1013 [00:00<?, ?it/s]

All files exist!


In [16]:

from itertools import groupby
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf

##############
# load model & audio and run audio through model
##############
import os
# Use old or new vocab?
# os.system("cp vocab_december.json vocab.json")
os.system("cp vocab_300_witn_numbers.json vocab.json")

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "/home/peterr/macocu/task8/", unk_token="[UNK]", 
    pad_token="[PAD]", word_delimiter_token=" ")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)


model_name = '/home/peterr/macocu/task8/38_unnormalised/checkpoint-5360'

model = Wav2Vec2ForCTC.from_pretrained(model_name).cuda()

def get_transcript(audio_filepath:str):
    speech, sample_rate = sf.read(audio_filepath)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()

    ##############
    # this is where the logic starts to get the start and end timestamp for each word
    ##############
    words = [w for w in transcription.split(' ') if len(w) > 0]
    predicted_ids = predicted_ids[0].tolist()
    duration_sec = input_values.shape[1] / sample_rate


    ids_w_time = [(i / len(predicted_ids) * duration_sec, _id) for i, _id in enumerate(predicted_ids)]
    # remove entries which are just "padding" (i.e. no characers are recognized)
    ids_w_time = [i for i in ids_w_time if i[1] != processor.tokenizer.pad_token_id]
    # now split the ids into groups of ids where each group represents a word
    split_ids_w_time = [list(group) for k, group
                        in groupby(ids_w_time, lambda x: x[1] == processor.tokenizer.word_delimiter_token_id)
                        if not k]
    assert len(split_ids_w_time) == len(words)  # make sure that there are the same number of id-groups as words. Otherwise something is wrong

    word_start_times = []
    word_end_times = []
    for cur_ids_w_time, cur_word in zip(split_ids_w_time, words):
        _times = [_time for _time, _id in cur_ids_w_time]
        word_start_times.append(min(_times))
        word_end_times.append(max(_times))

    return " ".join(words)

from tqdm.auto import tqdm
df["predictions"] = [get_transcript(path) for path in tqdm(df.path.values)]



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/1013 [00:00<?, ?it/s]

In [17]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

for splt in ["dev", "test"]:
    print("Evaluating on ", splt)
    wer = wer_metric.compute(
        references=df.loc[df.split==splt, "sentence"],
        predictions=df.loc[df.split==splt, "predictions"]
    )

    cer = cer_metric.compute(
        references=df.loc[df.split==splt, "sentence"],
        predictions=df.loc[df.split==splt, "predictions"]
    )

    print(f"{wer=:0.4f}, {cer=:0.4f}")

Evaluating on  dev
wer=0.1046, cer=0.0335
Evaluating on  test
wer=0.0761, cer=0.0234


In [ ]:
df[["sentence", "predictions", "split", "hashname"]].to_csv(
    "38_model_output_for_evaluation.csv", 
    index=False
    )